In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGANGP

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'LPMC'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,travel_mode,purpose,fueltype,faretype,bus_scale,travel_year,travel_month,travel_date,day_of_week,start_time_linear,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_n_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent
0,drive,HBO,Petrol_Car,child,0.0,2012,4,1,7,10.000000,...,0.134444,0.0,0.016667,0.000000,0,0.052222,0.0,0.14,0.0,0.111702
1,drive,HBO,Petrol_Car,free,0.0,2012,4,1,7,17.666667,...,0.241389,0.0,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
2,drive,HBO,Petrol_Car,full,1.0,2012,4,1,7,13.000000,...,0.257500,0.0,0.873889,0.089444,1,0.508333,3.0,1.59,0.0,0.356831
3,pt,HBW,Average_Car,full,1.0,2012,4,1,7,21.500000,...,0.123889,0.0,0.208056,0.091667,1,0.115556,3.0,0.33,0.0,0.033654
4,pt,HBO,Average_Car,free,0.0,2012,4,1,7,8.916667,...,0.171389,0.0,0.334444,0.000000,0,0.196389,0.0,0.53,0.0,0.035361


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "travel_month"),
        ("travel_date", "day_of_week"),
        ("travel_month", "travel_date"),
        ("travel_month", "driving_traffic_percent"),
        ("travel_month", "day_of_week"),
        ("travel_month", "travel_mode"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("day_of_week", "purpose"),
        ("day_of_week", "start_time_linear"),
        ("day_of_week", "travel_mode"),
        ("purpose", "distance"),
        ("purpose", "start_time_linear"),
        ("purpose", "travel_mode"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("start_time_linear", "travel_mode"),
        ("car_ownership", "fueltype"),
        ("car_ownership", "driving_license"),
        ("car_ownership", "travel_mode"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("female", "driving_license"),
        ("female", "travel_mode"),
        ("age", "bus_scale"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("age", "travel_mode"),
        ("driving_license", "travel_mode"),
        ("faretype", "cost_transit"),
        ("faretype", "bus_scale"),
        ("faretype", "travel_mode"),
        ("bus_scale", "cost_transit"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("distance", "pt_n_interchanges"),
        ("distance", "travel_mode"),
        ("pt_n_interchanges", "dur_pt_rail"),
        ("pt_n_interchanges", "dur_pt_bus"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("driving_traffic_percent", "travel_mode"),
        ("cost_driving_fuel", "cost_driving_con_charge"),
        ("cost_driving_fuel", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "cost_transit"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode"),
        ("cost_transit", "travel_mode")
    ])

In [9]:
output_folder = '../output/' + dataset + '/TEST5/'

In [10]:
datgan = DATWGANGP(continuous_columns, max_epoch=1000, batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1130 14:51:13 @DATSGAN.py:159] Found preprocessed data
[1130 14:51:13 @DATSGAN.py:167] Preprocessed data have been loaded!



[1130 14:51:13 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[1130 14:51:13 @DATSGANModel.py:213] Creating cell for travel_year (in-edges: 0; ancestors: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1130 14:51:13 @registry.py:126] gen/LSTM/travel_year/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1130 14:51:13 @registry.py:134] gen/LSTM/travel_year/FC output: [500, 50]
[1130 14:51:13 @registry.py:126] gen/LSTM/travel_year/FC_prob input: [500, 50]
[1130 14:51:13 @registry.py:134] gen/LSTM/

[1130 14:51:14 @registry.py:126] gen/LSTM/concat-driving_license/FC_inputs input: [500, 300]
[1130 14:51:14 @registry.py:134] gen/LSTM/concat-driving_license/FC_inputs output: [500, 100]
[1130 14:51:14 @registry.py:126] gen/LSTM/concat-driving_license/FC_states input: [500, 300]
[1130 14:51:14 @registry.py:134] gen/LSTM/concat-driving_license/FC_states output: [500, 100]
[1130 14:51:14 @registry.py:126] gen/LSTM/concat-driving_license/FC_h_outputs input: [500, 300]
[1130 14:51:14 @registry.py:134] gen/LSTM/concat-driving_license/FC_h_outputs output: [500, 100]
[1130 14:51:14 @registry.py:126] gen/LSTM/age-car_ownership-female/FC_noise input: [None, 600]
[1130 14:51:14 @registry.py:134] gen/LSTM/age-car_ownership-female/FC_noise output: [None, 200]
[1130 14:51:14 @registry.py:126] gen/LSTM/driving_license/FC input: [500, 100]
[1130 14:51:14 @registry.py:134] gen/LSTM/driving_license/FC output: [500, 50]
[1130 14:51:14 @registry.py:126] gen/LSTM/driving_license/FC_prob input: [500, 50]
[

[1130 14:51:14 @registry.py:126] gen/LSTM/concat-driving_traffic_percent/FC_states input: [500, 300]
[1130 14:51:14 @registry.py:134] gen/LSTM/concat-driving_traffic_percent/FC_states output: [500, 100]
[1130 14:51:14 @registry.py:126] gen/LSTM/concat-driving_traffic_percent/FC_h_outputs input: [500, 300]
[1130 14:51:14 @registry.py:134] gen/LSTM/concat-driving_traffic_percent/FC_h_outputs output: [500, 100]
[1130 14:51:14 @registry.py:126] gen/LSTM/driving_traffic_percent/FC input: [500, 100]
[1130 14:51:14 @registry.py:134] gen/LSTM/driving_traffic_percent/FC output: [500, 50]
[1130 14:51:14 @registry.py:126] gen/LSTM/driving_traffic_percent/FC_val input: [500, 50]
[1130 14:51:14 @registry.py:134] gen/LSTM/driving_traffic_percent/FC_val output: [500, 6]
[1130 14:51:14 @registry.py:126] gen/LSTM/driving_traffic_percent/FC_prob input: [500, 50]
[1130 14:51:14 @registry.py:134] gen/LSTM/driving_traffic_percent/FC_prob output: [500, 6]
[1130 14:51:14 @registry.py:126] gen/LSTM/driving_tr

[1130 14:51:15 @DATSGANModel.py:213] Creating cell for dur_pt_rail (in-edges: 2; ancestors: 7)
[1130 14:51:15 @registry.py:126] gen/LSTM/concat-dur_pt_rail/FC_inputs input: [500, 200]
[1130 14:51:15 @registry.py:134] gen/LSTM/concat-dur_pt_rail/FC_inputs output: [500, 100]
[1130 14:51:15 @registry.py:126] gen/LSTM/concat-dur_pt_rail/FC_states input: [500, 200]
[1130 14:51:15 @registry.py:134] gen/LSTM/concat-dur_pt_rail/FC_states output: [500, 100]
[1130 14:51:15 @registry.py:126] gen/LSTM/concat-dur_pt_rail/FC_h_outputs input: [500, 200]
[1130 14:51:15 @registry.py:134] gen/LSTM/concat-dur_pt_rail/FC_h_outputs output: [500, 100]
[1130 14:51:15 @registry.py:126] gen/LSTM/dur_pt_rail/FC input: [500, 100]
[1130 14:51:15 @registry.py:134] gen/LSTM/dur_pt_rail/FC output: [500, 50]
[1130 14:51:15 @registry.py:126] gen/LSTM/dur_pt_rail/FC_val input: [500, 50]
[1130 14:51:15 @registry.py:134] gen/LSTM/dur_pt_rail/FC_val output: [500, 6]
[1130 14:51:15 @registry.py:126] gen/LSTM/dur_pt_rail/FC


[1130 14:51:21 @logger.py:90] Argv: D:\Users\glede\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py -f C:\Users\glede\AppData\Roaming\jupyter\runtime\kernel-22c34e65-a7a1-4a18-8b28-d1b7b4ad1ff3.json



[1130 14:51:21 @model_utils.py:67] List of Trainable Variables: 
name                                                        shape          #elements
----------------------------------------------------------  -----------  -----------
gen/LSTM/f0-travel_year:0                                   [1, 100]             100
gen/LSTM/travel_year/LSTM_cell/kernel:0                     [500, 400]        200000
gen/LSTM/travel_year/LSTM_cell/bias:0                       [400]                400
gen/LSTM/travel_year/FC/W:0                                 [100, 50]           5000
gen/LSTM/travel_year/FC/b:0                                 [50]                  50
gen/LSTM/travel_year/FC_prob/W:0                            [50, 4]              200
gen/LSTM/travel_year/FC_prob/b:0         

...

[1130 15:20:12 @base.py:285] Epoch 999 (global_step 41957) finished, time:1.48 seconds.


[1130 15:20:12 @saver.py:79] Model saved to ../output/LPMC/TEST5/model\model-41957.
[1130 15:20:12 @monitor.py:467] GAN_loss/d_loss: -0.3088
[1130 15:20:12 @monitor.py:467] GAN_loss/g_loss: -1.569
[1130 15:20:12 @monitor.py:467] GAN_loss/gradient_penalty: 0.011309
[1130 15:20:12 @monitor.py:467] GAN_loss/gradient_rms: 1.009
[1130 15:20:12 @monitor.py:467] GAN_loss/kl_div: 0.11833
[1130 15:20:12 @monitor.py:467] QueueInput/queue_size: 50
[1130 15:20:12 @base.py:275] Start Epoch 1000 ...


100%|#####################################################################################|35/35[00:01<00:00,23.74it/s]

[1130 15:20:13 @base.py:285] Epoch 1000 (global_step 41999) finished, time:1.47 seconds.


[1130 15:20:14 @saver.py:79] Model saved to ../output/LPMC/TEST5/model\model-41999.
[1130 15:20:14 @monitor.py:467] GAN_loss/d_loss: -0.30195
[1130 15:20:14 @monitor.py:467] GAN_loss/g_loss: -1.5681
[1130 15:20:14 @monitor.py:467] GAN_loss/gradient_penalty: 0.011339
[1130 15:20:14 @monitor.py:467] GAN_loss/gradient_rms: 1.0101
[1130 15:20:14 @monitor.py:467] GAN_loss/kl_div: 0.12314
[1130 15:20:14 @monitor.py:467] QueueInput/queue_size: 50
[1130 15:20:14 @base.py:289] Training has finished!

[1130 15:20:14 @DATSGANModel.py:213] Creating cell for travel_year (in-edges: 0; ancestors: 0)
[1130 15:20:14 @DATSGANModel.py:213] Creating cell for car_ownership (in-edges: 0; ancestors: 0)
[1130 15:20:14 @DATSGANModel.py:213] Creating cell for female (in-edges: 0; ancestors: 0)
[1130 15:20:14 @DATSGANModel.py:213] Creating cell for age (in-edges: 0; ancestors: 0)
[1130 15:20:14 @DATSGANModel.py:213] Creating cell for travel_month (in-edges: 1; ancestors: 1)
[1130 15:20:14 @DATSGANModel.py:213] C

In [12]:
datgan.save('trained', force=True)

[1130 15:20:50 @DATSGAN.py:365] Model saved successfully.


In [13]:
beepy.beep(6)